In [1]:
%load_ext cython

In [10]:
%%cython -+ -I/usr/include -I/local/include -I../../../include -I../../../src -ltbb -lgomp -f --compile-args=-std=c++17 --compile-args=-fopenmp --compile-args=-g --compile-args=-O0
# distutils: language = c++

import numpy

cdef extern from "<bindings/cython/CyDisc.hpp>" namespace "sd::disc":
    dict cy_desc_from_matrix(x, y, int min_support, int is_sparse, int high_precision_float)
    dict cy_desc_from_transactions(x, y, int min_support, int is_sparse, int high_precision_float)

def desc_wrap(x, y=None, min_support = 2, is_sparse = False, high_precision_float = False):
    y = list(y) if y is not None else []
    if isinstance(x, list):
        if len(x) == 0:
            return []
        if not isinstance(x[0], list):
            raise Exception("I dont like that type")
        else:
            return cy_desc_from_transactions(x, y, min_support, is_sparse, high_precision_float)
    elif isinstance(x, numpy.ndarray):
        if x.ndim != 2: raise Exception("I dont like that type")
        if x.dtype != numpy.bool_:
            x = x.astype(numpy.bool_)
        return cy_desc_from_matrix(x, y, min_support, is_sparse, high_precision_float)
    else:
        raise Exception("I dont like that type")

In [3]:
# c = desc([[1,2,3],[1,2,3],[1],[1],[1]], [0,0,1,1,1])

In [32]:

# class Contrasting:
#     def shared(self, Y)
#     def shared(self, Y):
#         a_Y = [i for i in self.assignment[y] for y in Y]
#         return [self.summary[i] for i in a_Y if len(self.summary[i]) > 1] 
#     def characteristic(self, i):

class DESC:
    def __init__(self, min_support = 2, is_sparse = False, use_high_precision_float = False):
        self.min_support = min_support
        self.is_sparse = is_sparse
        self.use_high_precision_float = use_high_precision_float

    def fit(self, x, y=None):
        r = desc_wrap(x, y, self.min_support, self.is_sparse, self.use_high_precision_float)
        # clean singletons from output.
        # self.internal_state = r['internal_state']
        self.initial_objective = r['initial_objective']
        self.objective = r['objective']
        self.gain = self.objective[0] / self.initial_objective[0]
        self.elapsed_time_ms = r["elapsed_time[ms]"]
        self.summary = r['pattern_set']
        self.frequency = r['frequencies']
        if y is not None:
            self.assingment = r['assignment']

    # def predict(self, x):
    #     if self.internal_state is None:
    #         raise Exception("Cannot predict without fitting the model")
    #     return list(desc_classify_wrap(self.internal_state, x)) 

In [28]:
desc_wrap([[1,2]])

{'pattern_set': [[0], [1], [2]],
 'frequencies': [0.0, 1.0, 1.0],
 'initial_objective': (3.2034265038149176e-16, 3.2034265038149176e-16, 0.0),
 'objective': (3.2034265038149176e-16, 3.2034265038149176e-16, 0.0),
 'elapsed_time[ms]': 25}

In [29]:
desc = DESC()
desc.fit((numpy.random.uniform(size=(1000,2)) < 0.5))

In [33]:
desc.gain

1.0